Questo notebook integra i dati di ANAC con quelli di IndicePA e contiene una serie di analisi descrittive. 

Output: *data/anac/anac_indicepa.tsv* contiene tutti i bandi di gara avvenuti nel 2017 da parte di PA censite in indicePA.

**Note**
1. Sono stati estratti un totale di 3913440 gare, di cui 1842313 avvenute nell'anno 2017. Si procede all'analisi descrittiva di quest'ultimi bandi.
2. Molti data record contengono valori inconsistenti dovuti a typo (e.g. date con valori errate - 202/02/2016) o ad omissioni da parte delle PA (e.g. codici fiscali di strutture proponenti aventi valori nulli o uguali a *0*, codici fiscali di aggiudicatari offuscati o inesistenti). 


In [1]:
import pandas as pd
import numpy
import csv

In [2]:
f_anac = "data/anac/anacDataset_1522836870094_postprocessed2.tsv"
#df = pd.read_csv(f_anac, sep="\t", error_bad_lines=False, parse_dates=True)
df = pd.read_csv(f_anac, sep="\t", parse_dates=True)

/Users/fabiana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#ci sono typo in alcune date (e.g. 202/02/2016)
df['dataInizio'] = pd.to_datetime(df['dataInizio'], errors='coerce', format="%d/%m/%Y")
df['dataUltimazione'] = pd.to_datetime(df['dataUltimazione'], errors='coerce', format="%d/%m/%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3913440 entries, 0 to 3913439
Data columns (total 17 columns):
cig                                  object
cfStrutturaProponente                object
denominazioneStrutturaProponente     object
oggetto                              object
sceltaContraente                     object
importoAggiudicazione                float64
importoSommeLiquidate                float64
dataInizio                           datetime64[ns]
dataUltimazione                      datetime64[ns]
jsonPartecipanti                     object
jsonAggiudicatari                    object
totalePartecipanti                   int64
totaleAggiudicatari                  int64
totaleRaggruppamentopartecipanti     int64
totaleRaggruppamentoAggiudicatari    int64
cfPrimoaggiudicatario                object
denominazionePrimoaggiudicatario     object
dtypes: datetime64[ns](2), float64(2), int64(4), object(9)
memory usage: 507.6+ MB


In [4]:
df.head(3)

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
0,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Attrezzature per autoanalisi,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,3306.00,0.00,2016-11-25,2016-11-25,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",3,1,0,0,01615440342,Callegari Srl
1,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Cancelleria,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,2307.34,2307.34,2012-03-31,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,06450560963,CV office SAS di Del Freo Chiara E C
2,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Telefonia,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,4160.93,4160.93,2015-02-25,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",3,1,0,0,08539010010,Vodafone Italia SpA


## Analisi gare 2017

In [5]:
is_2017 = (df['dataInizio'] > '01/01/2017') & (df['dataInizio'] <= '31/12/2017')
df_2017 = df.loc[is_2017]
df_2017.shape

(1842313, 17)

In [6]:
df_cfPrimoaggiudicatario_count = df_2017[["cfStrutturaProponente", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"])["cfPrimoaggiudicatario"].agg(["count"]).reset_index(0)
#df_cfPrimoaggiudicatario_count.head(3)
df_cfPrimoaggiudicatario_sum = df_2017[["importoSommeLiquidate","importoAggiudicazione", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"]).sum().reset_index()
#df_cfPrimoaggiudicatario_sum.head(3)

In [7]:
print("df_cfPrimoaggiudicatario_count", df_cfPrimoaggiudicatario_count.shape)
print("df_cfPrimoaggiudicatario_sum", df_cfPrimoaggiudicatario_sum.shape)

df_cfPrimoaggiudicatario_count (315276, 2)
df_cfPrimoaggiudicatario_sum (315276, 3)


In [8]:
df_denominazione_aggiudicatario = df_2017[["cfPrimoaggiudicatario","denominazionePrimoaggiudicatario"]]
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario.set_index("cfPrimoaggiudicatario")
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
#df_denominazione_aggiudicatario.head(3)

In [9]:
df_join = df_cfPrimoaggiudicatario_count.set_index('cfPrimoaggiudicatario').join(df_cfPrimoaggiudicatario_sum.set_index('cfPrimoaggiudicatario'))
#df_join.head(3)
df_join.shape

(315276, 3)

In [10]:
#Elimino i duplicati dalle denominazioni
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
df_denominazione_aggiudicatario.shape

(315277, 1)

In [11]:
final_df = pd.merge(df_denominazione_aggiudicatario, df_join, left_index=True, right_index=True, how='inner')
final_df.shape

(315276, 4)

In [12]:
final_df.head(3)

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
09606050962,Kresco Group Multibrand srl,1,2.005000e+04,2.005000e+04
Raggruppamento,Raggruppamento,74668,1.290753e+09,5.298415e+09
01640430243,LIBRERIA PALAZZO ROBERTI,68,3.372552e+04,5.351900e+04


## Top aziende, nel 2017, aggiudicatare in termini di gare vinte e importo aggiudicazione 

**Note**

1. Gli aggiudicatari dei bandi di gara possono essere sia singole aziende che raggruppamenti. In quest'ultimo caso il campo *cfPrimoaggiudicatario* e il campo *denominazionePrimoaggiudicatario* avranno valore *Raggruppamento*;
2. Ci sono 74668 bandi di gara vinti da raggruppamenti. Si dedica a questi data record una successiva analisi.

In [13]:
top_aggiuficatari = final_df.sort_values(['count', "importoAggiudicazione"], ascending=False)
top_aggiuficatari.head()

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
Raggruppamento,Raggruppamento,74668,1.290753e+09,5.298415e+09
00150470342,Gruppo Spaggiari Parma SpA,14857,7.373984e+06,8.166934e+06
00488410010,TELECOM ITALIA SpA,10502,3.060910e+07,1.156872e+08
02027040019,CDATABORGIONE CENTRO DIDATTICO SRL,7522,3.413943e+06,3.970015e+06
03222970406,MyO Srl,7002,3.963308e+06,7.912744e+06


## Integrazione dati ANAC con IndicePA

Il file *"data/indicePA/indicePA.tsv"* è stato estratto a partire dal notebook *Analisi IndicePA*.

**Note**

- Anno 2017: Su 1.842.313 bandi di gara, solo 1.321.761 hanno codici fiscali di strutture proponenti validi. Questo è dovuto al fatto che alcuni codici fiscali sono errati, mentre altri sono codici fiscali di SPA (non censite in IndicePA). Il dataframe è salvato in *data/anac/anac_indicepa_2017.tsv*
- Tutti gli anni: Su 3.913.440  bandi di gara, 2.682.931 hanno codici fiscali di strutture proponenti validi.

In [14]:
f_indicePA = "data/indicePA/indicePA.tsv"
df_indicePA = pd.read_csv(f_indicePA, sep="\t")

In [15]:
df_indicePA = df_indicePA.rename(columns={'cf': 'cfStrutturaProponente'})
df_indicePA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13870 entries, 0 to 13869
Data columns (total 8 columns):
cfStrutturaProponente    13870 non-null int64
cod_amm                  13870 non-null object
regione                  13870 non-null object
provincia                13423 non-null object
comune                   13870 non-null object
indirizzo                13870 non-null object
tipologia_istat          13870 non-null object
tipologia_amm            13870 non-null object
dtypes: int64(1), object(7)
memory usage: 867.0+ KB


In [16]:
df_indicePA.head(2)

,cfStrutturaProponente,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,7640210964,amcma,Lombardia,MI,Cerro Maggiore,"Piazza Aldo Moro,1",Altri Enti Locali,Pubbliche Amministrazioni
1,80010060277,m_inf,Lazio,RM,Roma,"Piazzale Porta Pia, 1","Presidenza del Consiglio dei Ministri, Ministe...",Pubbliche Amministrazioni


### ANNO 2017

In [17]:
df_indicePA = df_indicePA.set_index("cfStrutturaProponente")
df_2017 = df_2017.set_index("cfStrutturaProponente")
print(df_indicePA.shape)
print(df_2017.shape)

(13870, 7)
(1842313, 16)


In [18]:
df_2017_indicePA = pd.merge(df_2017, df_indicePA, left_index=True, right_index=True, how='inner')
#df_2017_indicePA.shape

In [19]:
diff = df_2017.shape[0] - df_2017_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: ", diff)

Gare di PA non recensite su indicePA:  520552


In [20]:
df_2017_indicePA.head(2)

,cig,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,...,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
cfStrutturaProponente,,,,,,,,,,,,,,,,,,,,,
8010803,0000000000,COMUNE DI CINQUEFRONDI SERVIZI TECNICI,MANUTENZIONE RECINZIONE E MESSA IN SICUREZZA C...,06PROCEDURA NEGOZIATA SENZA PREVIA INDIZIONE D...,491.80,0.0,2017-12-20,NaT,"{""raggruppamento"":[],""partecipante"":[]}","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",...,0,Raggruppamento,Raggruppamento,c_c710,Calabria,RC,Cinquefrondi,Corso Garibaldi 103,Comuni e loro Consorzi e Associazioni,Pubbliche Amministrazioni
8010803,0000000000,COMUNE DI CINQUEFRONDI SERVIZI TECNICI,MATERIALE VARIO PER MANUTENZIONE VERDE PUBBLICO,17AFFIDAMENTO DIRETTO EX ART 5 DELLA LEGGE N38191,423.08,0.0,2017-12-20,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",...,0,00937680809,SARFSA,c_c710,Calabria,RC,Cinquefrondi,Corso Garibaldi 103,Comuni e loro Consorzi e Associazioni,Pubbliche Amministrazioni


In [21]:
df_2017_indicePA.to_csv("data/anac/anac_indicepa_2017.tsv", sep="\t")

### Tutti gli anni

In [22]:
df = df.reset_index().set_index("cfStrutturaProponente")
df_all_indicePA = pd.merge(df, df_indicePA, left_index=True, right_index=True, how='inner')
diff = df.shape[0] - df_all_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: ", diff)

Gare di PA non recensite su indicePA:  1230509


In [23]:
df_all_indicePA.to_csv("data/anac/anac_indicepa_all.tsv", sep="\t")